In [2]:
import pandas as pd

In [1]:
from caveclient import CAVEclient
client = CAVEclient()
auth = client.auth

In [3]:
# If first run uncomment to get instruction to get a token
#auth.get_new_token()
client.auth.token = "a68574bbbfddae1938daba7106d5035f"

In [4]:
# The state from https://github.com/NeLy-EPFL/neuromechfly-muscles-dev/issues/6#issuecomment-1881224757
muscles_state = client.state.get_state_json(5920404218576896)
anno_layers = [layer for layer in muscles_state['layers'] if layer['type'] == 'annotation']

In [5]:
fiber_lines = []
for layer in anno_layers:
    for anno in layer['annotations']:
        if anno['type'] != 'line':
            continue
        fiber_lines.append({
            #'id': anno['id'],
            'pointA': [int(i) for i in anno['pointA']],
            'pointB': [int(i) for i in anno['pointB']],
            'layer_name': layer['name'],
        })

In [6]:
fiber_lines = pd.DataFrame(fiber_lines)
# If layer name starts with LF, RF, LM, RM, LH, or RH, then it's a leg muscle
fiber_lines['is_leg_muscle'] = fiber_lines['layer_name'].str.startswith(('LF', 'RF', 'LM', 'RM', 'LH', 'RH'))
fiber_lines.loc[~fiber_lines.is_leg_muscle, 'leg'] = None
leg_fibers = fiber_lines[fiber_lines.is_leg_muscle].copy()
leg_fibers['leg'] = leg_fibers['layer_name'].str[0:2]
leg_fibers['origin-insertion'] = leg_fibers['layer_name'].str.split(' ').str[1]
leg_fibers['origin'] = leg_fibers['origin-insertion'].str[:2]
leg_fibers['insertion'] = leg_fibers['origin-insertion'].str[2:]

In [7]:
fiber_lines.head()

,pointA,pointB,layer_name,is_leg_muscle,leg
0,"[994, 797, 503]","[1732, 1262, 1081]",crazy proboscis muscle,False,None
1,"[999, 799, 528]","[1727, 1268, 1084]",crazy proboscis muscle,False,None
2,"[684, 1054, 725]","[671, 984, 2183]",L wing DLM,False,None
3,"[322, 1010, 1099]","[397, 1002, 1930]",L wing DLM,False,None
4,"[416, 1011, 950]","[476, 993, 2005]",L wing DLM,False,None
